## Dataset and Problem Introduction

In this analysis, we explore two different types of posts from Hacker News, a popular website where technology related posts are voted and commented upon.

Data Source: https://www.kaggle.com/hacker-news/hacker-news-posts/home
<br>Reference: https://www.datacamp.com/

## Reading the data and removing headers

In [1]:
# Read in the data.
import csv
f = open('datasets/hacker_news.csv')
hn = list(csv.reader(f))

# Remove the headers.
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We can see above that the dataset contains the title of the posts, the number of comments for each post, and the date the post was created.

The two types of news posts we want to explore begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?". While users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting.

We want to understand the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?
- Is there any relation between the title of a post and the number of comments received?

<b>Note:</b> The dataset used is reduced from 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

# Extracting Ask HN and Show HN Posts

In [2]:
# Identify posts that begin with either `Ask HN` or `Show HN` and separate the data into different lists.
ask_posts = []
show_posts =[]
other_posts = []

for post in hn:
    title = post[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(post)
    elif title.lower().startswith("show hn"):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
print("Ask HN: "+str(len(ask_posts)))
print("Show HN: "+str(len(show_posts)))
print("Other: "+str(len(other_posts)))

Ask HN: 1744
Show HN: 1162
Other: 17194


# Calculating the Average Number of Comments for Ask HN and Show HN Posts

In [3]:
# Calculate the average number of comments `Ask HN` posts receive.
total_ask_comments = 0

for post in ask_posts:
    total_ask_comments += int(post[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print("Average Ask HN Comments: "+str(avg_ask_comments))

total_show_comments = 0

for post in show_posts:
    total_show_comments += int(post[4])
    
avg_show_comments = total_show_comments / len(show_posts)
print("Average Show HN Comments: "+str(avg_show_comments))

Average Ask HN Comments: 14.038417431192661
Average Show HN Comments: 10.31669535283993


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we focus our remaining analysis on these posts. 

# Amount of Ask Posts Created per Hour

In [4]:
# Calculate the amount of ask posts created during each hour of day and the number of comments received.
import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append(
        [post[6], int(post[4])]
    )

comments_by_hour = {}
counts_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for each_row in result_list:
    date = each_row[0]
    comment = each_row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time in counts_by_hour:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1
    else:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1

print('Number of posts per hour:')
print(counts_by_hour)
print('Number of comments on posts per hour:')
print(comments_by_hour)

Number of posts per hour:
{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}
Number of comments on posts per hour:
{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


## Average Number of Comments for Ask HN Posts by Hour

In [5]:
# Calculate the average amount of comments `Ask HN` posts created at each hour of the day receive.
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

print(sorted(avg_by_hour))

[['00', 8.127272727272727], ['01', 11.383333333333333], ['02', 23.810344827586206], ['03', 7.796296296296297], ['04', 7.170212765957447], ['05', 10.08695652173913], ['06', 9.022727272727273], ['07', 7.852941176470588], ['08', 10.25], ['09', 5.5777777777777775], ['10', 13.440677966101696], ['11', 11.051724137931034], ['12', 9.41095890410959], ['13', 14.741176470588234], ['14', 13.233644859813085], ['15', 38.5948275862069], ['16', 16.796296296296298], ['17', 11.46], ['18', 13.20183486238532], ['19', 10.8], ['20', 21.525], ['21', 16.009174311926607], ['22', 6.746478873239437], ['23', 7.985294117647059]]


## Sorting Values

In [6]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

# Sort the values and print the the 5 hours with the highest average comments.

print("Top Hours for 'Ask HN' Comments")
for avg, hr in sorted_swap[:24]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )

Top Hours for 'Ask HN' Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post
13:00: 14.74 average comments per post
10:00: 13.44 average comments per post
14:00: 13.23 average comments per post
18:00: 13.20 average comments per post
17:00: 11.46 average comments per post
01:00: 11.38 average comments per post
11:00: 11.05 average comments per post
19:00: 10.80 average comments per post
08:00: 10.25 average comments per post
05:00: 10.09 average comments per post
12:00: 9.41 average comments per post
06:00: 9.02 average comments per post
00:00: 8.13 average comments per post
23:00: 7.99 average comments per post
07:00: 7.85 average comments per post
03:00: 7.80 average comments per post
04:00: 7.17 average comments per post
22:00: 6.75 average comments per post
09:00: 5.58 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

## Relation between authors and the number of comments

In [7]:
# Creating a Author - Comment distribution

authors={}                         
for row in ask_posts:               
    name=row[5]                      
    comment=int(row[4])               
    if name in authors:                
        authors[name] += comment        
    else:                                
        authors[name] = comment           

# Creating a list with author names and comments
author_list=[]                                 

for name in authors:                             
    author_list.append([authors[name], name])     

sorted_list=sorted(author_list, reverse=True)
print("Top 10 authors with the most comments:")
print(sorted_list[:10])                                

Top 10 authors with the most comments:
[[3046, 'whoishiring'], [868, 'mod50ack'], [691, 'boren_ave11'], [531, 'schappim'], [520, 'sama'], [489, 'prmph'], [383, 'Apocryphon'], [309, 'curiousgal'], [284, 'mikemajzoub'], [258, 'philippnagel']]


## Titles and corresponding comments for top the author

In [8]:
# Printing the details of the author with highest comments
for row in ask_posts:                                                    
    if (row[5]=='whoishiring'):                         
        print( "Title :",row[1], "\n", "No. of comments: ", row[4])  

Title : Ask HN: Who wants to be hired? (June 2016) 
 No. of comments:  250
Title : Ask HN: Freelancer? Seeking freelancer? (December 2015) 
 No. of comments:  93
Title : Ask HN: Who is hiring? (September 2016) 
 No. of comments:  910
Title : Ask HN: Who wants to be hired? (August 2016) 
 No. of comments:  118
Title : Ask HN: Freelancer? Seeking freelancer? (September 2016) 
 No. of comments:  85
Title : Ask HN: Who is hiring? (August 2016) 
 No. of comments:  947
Title : Ask HN: Who wants to be hired? (April 2016) 
 No. of comments:  283
Title : Ask HN: Freelancer? Seeking freelancer? (November 2015) 
 No. of comments:  158
Title : Ask HN: Who wants to be hired? (March 2016) 
 No. of comments:  202


# Conclusion

In this project, we analysed ask posts and show posts to determine which type of post and time received the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we recommend the post be categorised as an ask post and created between 15:00 and 16:00. 

However, it should be noted that the dataset we analysed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 received the most comments on average. 

For the top author's (whoishiring) posts, we note that by consistently posting about hiring related questions, this attracted a lot of comments, so talking about the recruitment process could lead to more activity on a post.